In [ ]:
from munch import Munch
from tf.app import use

In [ ]:
B = use("etcbc/dss", hoist=globals())
DSS = Munch({"F": F, "L": L, "T": T, "name": "DSS"})

A = use("etcbc/bhsa", hoist=globals())
BHSA = Munch({"F": F, "L": L, "T": T, "name": "BHSA"})

In [ ]:
del F, L, T

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import collections
from itertools import chain
from collections import defaultdict

In [ ]:
dss_sections = {}
for word in DSS.F.otype.s("word"):    
    scroll = DSS.T.scrollName(DSS.L.u(word, "scroll")[0])
    book = DSS.F.book_etcbc.v(word)    
    chapter = DSS.F.chapter.v(word)
    verse = DSS.F.verse.v(word)    
    if None in (scroll, book, chapter, verse):
        continue
    section = (book, chapter, verse)
    dss_sections.setdefault(section, {}).setdefault(scroll, []).append(word)

In [ ]:
def remove_duplicates(iterable):
    return list(dict.fromkeys(iterable))


class TFOb:
    def __init__(self, ids, source):
        if type(ids) is int:
            ids = [ids]
        self.ids = remove_duplicates(ids)
        self.source = source
        self._levels = ["to_" + otype + "s" for otype in source.F.otype.all]
        
    @classmethod
    def section(self, section, source, scroll=None):
        if source.name == "BHSA":
            return TFOb(source.T.nodeFromSection(section), source)
        
        section = (section[0], str(section[1]), str(section[2]))
        dss_section = dss_sections[section]
        if scroll is None:
            scroll = list(dss_section.keys())[0]
        return TFOb(dss_section[scroll], source)
        
    def __getattr__(self, attr):
        if attr in self._levels:
            level = "to_" + self.level + "s"
            if self.level == "none":
                return self
            level_index = self._levels.index(level)
            new_level_index = self._levels.index(attr)
            if new_level_index > level_index:
                return self.down(attr[3:-1])
            elif new_level_index < level_index:
                return self.up(attr[3:-1])
            else:
                return self
            
        if self.source.name == "DSS" and attr == "lex":
            attr = "lex_etcbc"
            
        feature = getattr(self.source.F, attr)
        return [getattr(self.source.F, attr).v(id_) for id_ in self.ids]
    
    def copy(self):
        return TFOb(self.ids.copy(), source)

    def up(self, otype=None):
        if self.level in (otype, "none"):
            return self
        return TFOb(
            chain(*[self.source.L.u(id_, otype) for id_ in self.ids]),
            source=self.source,
        )

    def down(self, otype=None):
        if self.level in (otype, "none"):
            return self
        return TFOb(
            chain(*[self.source.L.d(id_, otype) for id_ in self.ids]),
            source=self.source,
        )

    def filter(self, **kwargs):
        ids = []
        for id_ in self.ids:
            for key, value in kwargs.items():
                if key == "lex" and self.source.name == "DSS":
                    key = "lex_etcbc"
                if getattr(self.source.F, key).v(id_) != value:
                    break
            else:
                ids.append(id_)
        return TFOb(ids, source=self.source)

    def first(self, **kwargs):
        ids = []
        for id_ in self.ids:
            for key, value in kwargs.items():
                if key == "lex" and self.source.name == "DSS":
                    key = "lex_etcbc"
                if getattr(self.source.F, key).v(id_) != value:
                    break
            else:
                return TFOb(id_, source=self.source)
        return TFOb([], source=self.source)
            
    def __getitem__(self, i):
        return TFOb(self.ids[i], source=self.source)
    
    def __len__(self):
        return len(self.ids)

    @property
    def text(self):
        return self.source.T.text(self.ids)
    
    def __str__(self):
        if self.level not in ("word", "none"):
            return str(self.down("word"))
        return " ".join([g_cons for g_cons in self.g_cons if g_cons])
    
    def __dir__(self):
        return list(self.__dict__.keys()) + dir(self.source.F) + self._levels

    @property
    def level(self):
        if len(self.ids) > 0:
            return self.otype[0]
        return "none"

    def __repr__(self):
        level = self.level
        if level != "none":
            level += "s"
        return f'<{self.level}_{len(self)} "{str(self)}">'

In [ ]:
TFOb.section(["Isaiah", 6, 6], BHSA).to_words

In [ ]:
TFOb.section(["Isaiah", 6, 6], BHSA).to_words.g_cons

In [ ]:
TFOb.section(["Genesis", 1, 1], BHSA).to_words.first(lex="BR>[").to_clauses.to_phrases.first(function="Subj")

In [ ]:
verb = TFOb.section(["Genesis", 1, 1], BHSA).to_words.first(lex="BR>[")
verb.to_clauses.to_phrases.first(function="Subj")

In [ ]:
verb.to_words

In [ ]:
verb = TFOb.section(["Genesis", 1, 1], DSS).down("word").first(lex="BR>[")
verb.up("clause").down("phrase").first(function="Subj")

In [ ]:
TFOb.section(["Genesis", 1, 1], BHSA).down("word").first(lex="BR>[")

In [ ]:
TFOb.section(["Genesis", 1, 1], BHSA).down("word").lex

In [ ]:
TFOb.section(["Isaiah", 20, 6], DSS)

In [ ]:
TFOb.section(["Isaiah", 6, 6], BHSA)

In [ ]:
TFOb.section(["Isaiah", 6, 6], BHSA).down("word").lex

In [ ]:
word = TFOb(1033, DSS)
word

In [ ]:
TFOb(1, BHSA).up("clause").down("phrase").first(function="Subj")

In [ ]:
word.up("clause").down("phrase")

In [ ]:
word.up("clause").down("phrase")[0]

In [ ]:
clause.down("phrase").filter(function="toto")

In [ ]:
phrases.function

In [ ]:
def verse_lex(section):
    verse = (T.nodeFromSection(section))
    return [F.lex.v(w) for w in L.d(verse, "word")]

In [ ]:
verse_lex(("Genesis", 12, 6))

In [ ]:
A.indent(reset=True)
A.info("Counting nodes ...")

i = 0
for n in N.walk():
    i += 1

A.info("{} nodes".format(i))

In [ ]:
F.otype.slotType

In [ ]:
A.indent(reset=True)
A.info("counting objects ...")

for otype in F.otype.all:
    i = 0

    A.indent(level=1, reset=True)

    for n in F.otype.s(otype):
        i += 1

    A.info("{:>7} {}s".format(i, otype))

A.indent(level=0)
A.info("Done")

In [ ]:
verbs = collections.Counter()
A.indent(reset=True)
A.info("Collecting data")

lines = []
n = 0
for w in N.walk():
    # for w in F.otype.s("lex"):
    if F.lex.v(w) == "BW>[" and F.sp.v(w) == "verb" and F.otype.v(w) == "word":
        n += 1
        if n <= 3:
            continue
        section = T.sectionFromNode(w)
        lines.append(
            {
                "node": w,
                "tense": F.vt.v(w),
                "stem": F.st.v(w),
                "book": section[0],
                "chapter": section[1],
                "verse": section[2],
            }
        )
        verse = L.u(w, "verse")
        print("verse", verse, T.text(verse))
        
        clause = L.u(w, "clause")[0]
        print("clause", clause, T.text(clause))
        sentence = L.u(w, "sentence")
        print("sentence", sentence, T.text(sentence))

        clause_phrases = L.d(clause, "phrase")
        print("clause_phrases", clause_phrases)
        
        subject = None
        for phrase in clause_phrases:
            if F.function.v(phrase) == "Subj":
                subject = phrase
                break
                
        print("subject", subject, T.text(subject))
        
        cmpls = [phrase for phrase in clause_phrases if F.function.v(phrase) == "Cmpl"]
        
        print("Complement", cmpls, [T.text(cmpl) for cmpl in cmpls])
 
            
        for word in clause_words:
            print(F.sp.v(word))
        # L.d(clause, sp="verb")
        # print("subject", subject, T.text(subject))
        break
        # print(T.sectionFromNode(w))
        # print("book", F.book.v(w))
        # print(F.lex_utf8.v(w))
        # print(F.vt.v(w))
        # print(F.vs.v(w))

A.info("Done")

In [ ]:
F.lex

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(lines)
df

In [ ]:
df.to_csv("BO.csv", index=False)

In [ ]:
Fall()

In [ ]:
for name in Fall():
    print(name, getattr(F, name).v(1033))

In [ ]:
F.st.v(935)

In [ ]:
getattr(F, "st").v(935)

In [ ]:
for name in Fall():
    print(name, getattr(F, name).v(1437760))
    print(name, getattr(F, name).v(935))